## Task 2: Tensorflow (Bonus)
**Description:** Prepare a Neural Network Model for the case above using Tensorflow. You can write your own code [bonus] or use snippets from any solutions found online.

In [1]:
import numpy as np

import tensorflow as tf
from keras.models import Model
from keras.layers import Layer, Input

### Multilayer Preceptron

In [2]:
# Linear class for mlp
class Linear(Layer):
  def __init__(self, units=2, **kwargs):
    super(Linear, self).__init__(**kwargs)
    self.units = units 

  def build(self, input_shape):
    self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
    self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

  def get_config(self): 
    config = super(Linear, self).get_config()
    config.update({'units': self.units})
    return config

In [3]:
# mlp class to develop neural network
class MLP(Layer):
    def __init__(self):
        super(MLP, self).__init__()
        self.linear_1 = Linear(3)
        self.linear_2 = Linear(2)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.tanh(x)
        x = self.linear_2(x)
        x = tf.nn.sigmoid(x)
        return x

    def build_graph(self, raw_shape):
        x = Input(shape=raw_shape)
        return Model(inputs=[x], outputs=self.call(x))

In [4]:
def train_step(step, x, y):
    ''' input: x, y <- typically batches 
        input: step <- batch step
        return: loss values'''
   
    with tf.GradientTape() as tape:
        logits = mlp(x, training=True) # forward pass
        train_loss_value = loss_fn(y, logits) # loss function

    # compute gradient for backpropagation
    grads = tape.gradient(train_loss_value, mlp.trainable_weights)

    # update weights
    optimizer.apply_gradients(zip(grads, mlp.trainable_weights))

    # update metric
    train_acc_metric.update_state(y, logits)

    return train_loss_value

In [5]:
def test_step(step, x, y):
    ''' input: x, y <- typically batches 
    input: step <- batch step
    return: loss value '''

    # forward pass, no backpropagation
    val_logits = mlp(x, training=False) 

    # Compute the loss value 
    val_loss_value = loss_fn(y, val_logits)

    # Update val metric
    val_acc_metric.update_state(y, val_logits)


    return val_loss_value

### Data

In [6]:
# dummy data
np.random.seed(1)
x_train = np.random.randint(10,size=(10,2))*np.ones([10,2])
y_train = np.random.randint(10,size=(10,2))*np.ones([10,2])

x_test = np.random.randint(10,size=(5,2))*np.ones([5,2])
y_test = np.random.randint(10,size=(5,2))*np.ones([5,2])

In [7]:
# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(32)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_dataset = val_dataset.batch(32)

2022-03-16 16:19:49.517009: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Training and Validating

In [8]:
mlp = MLP() # initialize model

In [9]:
# Initial parameters
optimizer = tf.keras.optimizers.SGD() # Stochastic Gradient Descent

# initiate performance metrics
loss_fn = tf.keras.losses.MeanSquaredError() 
train_acc_metric = tf.keras.metrics.Accuracy()
val_acc_metric   = tf.keras.metrics.Accuracy()

In [10]:
# training loop 
for epoch in range(20):
    # Iterate over the batches of the train dataset.
    for train_batch_step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        train_batch_step = tf.convert_to_tensor(train_batch_step, dtype=tf.int64)
        train_loss_value = train_step(train_batch_step, 
                                      x_batch_train, y_batch_train)

    # evaluation on validation set 
    for test_batch_step, (x_batch_val, y_batch_val) in enumerate(val_dataset):
        test_batch_step = tf.convert_to_tensor(test_batch_step, dtype=tf.int64)
        val_loss_value = test_step(test_batch_step, x_batch_val, y_batch_val)


    template = 'epoch: {} loss: {}  acc: {} validation_loss: {} val acc: {}\n'
    print(template.format(
        epoch + 1,
        train_loss_value, (train_acc_metric.result()),
        val_loss_value, float(val_acc_metric.result())
    ))
        
    # Reset metrics at the end of each epoch
    train_acc_metric.reset_states()
    val_acc_metric.reset_states()

epoch: 1 loss: 34.4508056640625  acc: 0.0 validation_loss: 37.41999816894531 val acc: 0.0

epoch: 2 loss: 34.405364990234375  acc: 0.0 validation_loss: 37.37744903564453 val acc: 0.0

epoch: 3 loss: 34.35980224609375  acc: 0.0 validation_loss: 37.33461380004883 val acc: 0.0

epoch: 4 loss: 34.31391143798828  acc: 0.0 validation_loss: 37.29127883911133 val acc: 0.0

epoch: 5 loss: 34.26748275756836  acc: 0.0 validation_loss: 37.24724578857422 val acc: 0.0

epoch: 6 loss: 34.22031784057617  acc: 0.0 validation_loss: 37.20232391357422 val acc: 0.0

epoch: 7 loss: 34.1722412109375  acc: 0.0 validation_loss: 37.156349182128906 val acc: 0.0

epoch: 8 loss: 34.123077392578125  acc: 0.0 validation_loss: 37.109169006347656 val acc: 0.0

epoch: 9 loss: 34.072731018066406  acc: 0.0 validation_loss: 37.060691833496094 val acc: 0.0

epoch: 10 loss: 34.021087646484375  acc: 0.0 validation_loss: 37.01082229614258 val acc: 0.0

epoch: 11 loss: 33.968116760253906  acc: 0.0 validation_loss: 36.959514617